#### MP based asynchronous testing

In [4]:
import multiprocessing as mp
import TestPool_Unit
from shutil import copyfile
import numpy as np

In [3]:
def Evaluate(result_arr):
    print('Files Processed: ', len(result_arr))
    recalls = []
    recalls_of_word = []
    precisions = []
    precisions_of_words = []
    fully_Correct_l = 0
    fully_Correct_w = 0
    for entry in result_arr:
        (word_match, lemma_match, n_dcsWords, n_output_nodes) = entry
        recalls.append(lemma_match/n_dcsWords)
        recalls_of_word.append(word_match/n_dcsWords)

        precisions.append(lemma_match/n_output_nodes)
        precisions_of_words.append(word_match/n_output_nodes)
        if lemma_match == n_dcsWords:
            fully_Correct_l += 1
        if word_match == n_dcsWords:
            fully_Correct_w += 1
    print('Avg. Micro Recall of Lemmas: {}'.format(np.mean(np.array(recalls))))
    print('Avg. Micro Recall of Words: {}'.format(np.mean(np.array(recalls_of_word))))
    print('Avg. Micro Precision of Lemmas: {}'.format(np.mean(np.array(precisions))))
    print('Avg. Micro Precision of Words: {}'.format(np.mean(np.array(precisions_of_words))))
    print('Fully Correct Lemmawise: {}'.format(fully_Correct_l/len(recalls_of_word)))
    print('Fully Correct Wordwise: {}'.format(fully_Correct_w/len(recalls_of_word)))

In [11]:
modelFile = 'outputs/train_t1631702295635/nnet_i300.p' #margin_f
_dump = True
if _dump:
    _outFile = 'outputs/dump_predictions/pred_1L_bigram'
else:
    _outFile = None
# Backup the model file
copyfile(modelFile, modelFile + '.bk')

# Create Queue, Result array
queue = mp.Queue()
result_arr = []

# Start 6 workers - 8 slows down the pc
proc_count = 4
procs = [None]*proc_count
for i in range(proc_count):
    vpid = i
    procs[i] = mp.Process(target = TestPool_Unit.pooled_Test, args = \
                          (modelFile, vpid, queue, '../NewData/skt_dcs_DS.bz2_10K/', 2400, _dump, _outFile))
# Start Processes
for i in range(proc_count):
    procs[i].start()

In [14]:
# Fetch partial results
while not queue.empty():
    result_arr.append(queue.get())
# Evaluate results till now
Evaluate(result_arr)

Files Processed:  0
Avg. Micro Recall of Lemmas: nan
Avg. Micro Recall of Words: nan
Avg. Micro Precision of Lemmas: nan
Avg. Micro Precision of Words: nan


d:\program files\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
d:\program files\anaconda3\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


ZeroDivisionError: division by zero

In [15]:
# Check status
for i in range(proc_count):
    p = procs[i]
    print('Process with\t vpid: {}\t ->\t pid: {}\t ->\t running status: {}'.format(i, p.pid, p.is_alive()))

Process with	 vpid: 0	 ->	 pid: 12576	 ->	 running status: False
Process with	 vpid: 1	 ->	 pid: 7192	 ->	 running status: False
Process with	 vpid: 2	 ->	 pid: 4472	 ->	 running status: False
Process with	 vpid: 3	 ->	 pid: 8188	 ->	 running status: False


In [152]:
# Properly Join
for i in range(proc_count):
    procs[i].join()

In [13]:
# Force Terminate
for p in procs:
    p.terminate()

In [865]:
np.mean([a[2] for a in result_arr])

8.3328244274809169